In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
file_path = 'Linear_reg_test_data.csv'
df = pd.read_csv(file_path)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   LOB          55 non-null     object 
 1   origin       55 non-null     object 
 2   valuation    55 non-null     object 
 3   development  55 non-null     int64  
 4   CumPaidLoss  55 non-null     int64  
 5   Ultimate     55 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.7+ KB


In [4]:
df.head()

,LOB,origin,valuation,development,CumPaidLoss,Ultimate
0,comauto,1/1/1988,12/31/1988,12,154058,626097.0
1,comauto,1/1/1988,12/31/1989,24,326916,626097.0
2,comauto,1/1/1988,12/31/1990,36,447963,626097.0
3,comauto,1/1/1988,12/31/1991,48,528167,626097.0
4,comauto,1/1/1988,12/31/1992,60,574471,626097.0


In [5]:
date_cols = ['origin', 'valuation']
for column in date_cols:
    df[column] = pd.to_datetime(df[column])

In [6]:
current_eval = '12/31/1997'
prediction_data = df[df['valuation'] == current_eval]

In [7]:
fitting_data = df[df['valuation'] != current_eval]

In [8]:
models = {}
predictions = []

In [9]:
for (lob, development), group in fitting_data.groupby(['LOB', 'development']):
    if group.shape[0] > 1:
        X = group[['CumPaidLoss']]
        y = group['Ultimate']

        #fit a linear regression model
        model = LinearRegression()
        model.fit(X, y)

        #Store the model for reference
        models[(lob, development)] = model

        target_subset = prediction_data[(prediction_data['LOB'] == lob) & (prediction_data['development'] == development)].copy()
        if not target_subset.empty:
            X_new = target_subset[['CumPaidLoss']]
            target_subset['Least_Squares_Ult'] = model.predict(X_new)
            predictions.append(target_subset)

predictions_df = pd.concat(predictions, ignore_index=True)


In [10]:
predictions_df

,LOB,origin,valuation,development,CumPaidLoss,Ultimate,Least_Squares_Ult
0,comauto,1997-01-01,1997-12-31,12,272342,9.041841e+05,1.131940e+06
1,comauto,1996-01-01,1997-12-31,24,510191,1.027631e+06,1.141180e+06
2,comauto,1995-01-01,1997-12-31,36,675166,1.096179e+06,1.158015e+06
3,comauto,1994-01-01,1997-12-31,48,768095,1.098775e+06,1.142803e+06
4,comauto,1993-01-01,1997-12-31,60,762039,1.000256e+06,1.027862e+06
5,comauto,1992-01-01,1997-12-31,72,731033,9.102624e+05,8.983256e+05
6,comauto,1991-01-01,1997-12-31,84,711762,8.447233e+05,8.194450e+05
7,comauto,1990-01-01,1997-12-31,96,718396,8.140462e+05,8.569705e+05
